In [1]:
import numpy as np
import pandas as pd
import os
import argparse
import errno
import scipy.misc
import dlib
import cv2
from skimage.feature import hog
import matplotlib.pyplot as plt
from PIL import Image
from parameters import DATASET, NETWORK


In [2]:
import keras
from keras.models import Sequential
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import  BatchNormalization
from keras.layers.convolutional import *
from keras import regularizers
import tensorflow as tf


Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13806964906678030413
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 6735474197
locality {
  bus_id: 1
}
incarnation: 17918161769655793941
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0"
]


In [4]:
# initialization
image_height = 48
image_width = 48
window_size = 24
window_step = 6
ONE_HOT_ENCODING = False
SAVE_IMAGES = False
GET_LANDMARKS = True
GET_HOG_FEATURES = True
GET_HOG_IMAGES = True
GET_HOG_WINDOWS_FEATURES = True
SELECTED_LABELS = []

expressions = [0,1,2,3,4,5,6]
for i in range(0,len(expressions)):
    label = int(expressions[i])
    if (label >=0 and label<=6 ):
        SELECTED_LABELS.append(label)
        
if SELECTED_LABELS == []:
    SELECTED_LABELS = [0,1,2,3,4,5,6]
print( str(len(SELECTED_LABELS)) + " expressions")

IMAGES_PER_LABEL = 10000
OUTPUT_FOLDER_NAME = "fer2013"




7 expressions


In [5]:
############## Load  Data ######################
def load_data(validation=False, test=False):
    
    data_dict = dict()
    validation_dict = dict()
    test_dict = dict()

    if DATASET.name == "Fer2013":

        # load train set
        data_dict['X'] = np.load(DATASET.train_folder + '/images.npy')
        data_dict['X'] = data_dict['X'].reshape([-1, NETWORK.input_size, NETWORK.input_size, 1])
        if NETWORK.use_landmarks:
            data_dict['X2'] = np.load(DATASET.train_folder + '/landmarks.npy')
        if NETWORK.use_hog_and_landmarks:
            data_dict['X2'] = np.load(DATASET.train_folder + '/landmarks.npy')
            data_dict['X2'] = np.array([x.flatten() for x in data_dict['X2']])
            data_dict['X2'] = np.concatenate((data_dict['X2'], np.load(DATASET.train_folder + '/hog_features.npy')), axis=1)
        data_dict['Y'] = np.load(DATASET.train_folder + '/labels.npy')
        if DATASET.trunc_trainset_to > 0:
            data_dict['X'] = data_dict['X'][0:DATASET.trunc_trainset_to, :, :]
            if NETWORK.use_landmarks and NETWORK.use_hog_and_landmarks:
                data_dict['X2'] = data_dict['X2'][0:DATASET.trunc_trainset_to, :]
            elif NETWORK.use_landmarks:
                data_dict['X2'] = data_dict['X2'][0:DATASET.trunc_trainset_to, :, :]
            data_dict['Y'] = data_dict['Y'][0:DATASET.trunc_trainset_to, :]

        if validation:
            # load validation set
            validation_dict['X'] = np.load(DATASET.validation_folder + '/images.npy')
            validation_dict['X'] = validation_dict['X'].reshape([-1, NETWORK.input_size, NETWORK.input_size, 1])
            if NETWORK.use_landmarks:
                validation_dict['X2'] = np.load(DATASET.validation_folder + '/landmarks.npy')
            if NETWORK.use_hog_and_landmarks:
                validation_dict['X2'] = np.load(DATASET.validation_folder + '/landmarks.npy')
                validation_dict['X2'] = np.array([x.flatten() for x in validation_dict['X2']])
                validation_dict['X2'] = np.concatenate((validation_dict['X2'], np.load(DATASET.validation_folder + '/hog_features.npy')), axis=1)
            validation_dict['Y'] = np.load(DATASET.validation_folder + '/labels.npy')
            if DATASET.trunc_validationset_to > 0:
                validation_dict['X'] = validation_dict['X'][0:DATASET.trunc_validationset_to, :, :]
                if NETWORK.use_landmarks and NETWORK.use_hog_and_landmarks:
                    validation_dict['X2'] = validation_dict['X2'][0:DATASET.trunc_validationset_to, :]
                elif NETWORK.use_landmarks:
                    validation_dict['X2'] = validation_dict['X2'][0:DATASET.trunc_validationset_to, :, :]
                validation_dict['Y'] = validation_dict['Y'][0:DATASET.trunc_validationset_to, :]
        
        if test:
            # load test set
            test_dict['X'] = np.load(DATASET.test_folder + '/images.npy')
            test_dict['X'] = test_dict['X'].reshape([-1, NETWORK.input_size, NETWORK.input_size, 1])
            if NETWORK.use_landmarks:
                test_dict['X2'] = np.load(DATASET.test_folder + '/landmarks.npy')
            if NETWORK.use_hog_and_landmarks:
                test_dict['X2'] = np.load(DATASET.test_folder + '/landmarks.npy')
                test_dict['X2'] = np.array([x.flatten() for x in test_dict['X2']])
                test_dict['X2'] = np.concatenate((test_dict['X2'], np.load(DATASET.test_folder + '/hog_features.npy')), axis=1)
            test_dict['Y'] = np.load(DATASET.test_folder + '/labels.npy')
            if DATASET.trunc_testset_to > 0:
                test_dict['X'] = test_dict['X'][0:DATASET.trunc_testset_to, :, :]
                if NETWORK.use_landmarks and NETWORK.use_hog_and_landmarks:
                    test_dict['X2'] = test_dict['X2'][0:DATASET.trunc_testset_to, :]
                elif NETWORK.use_landmarks:
                    test_dict['X2'] = test_dict['X2'][0:DATASET.trunc_testset_to, :, :]
                test_dict['Y'] = test_dict['Y'][0:DATASET.trunc_testset_to, :]

        if not validation and not test:
            return data_dict
        elif not test:
            return data_dict, validation_dict
        else: 
            return data_dict, validation_dict, test_dict
    else:
        print( "Unknown dataset")
        exit()

In [6]:
data, validation, test = load_data(validation=True, test= True )

In [7]:
data['X'].shape



(28709, 48, 48, 1)

In [8]:
print(data['Y'].shape)
print(data['Y'])



(28709, 7)
[[1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 0 0]
 [1 0 0 ..., 0 0 0]
 [0 0 0 ..., 1 0 0]]


In [9]:
## parameter :
input_size = 48
Landmasks_size = data['X2'].shape[1]
batch_size = 8
no_training = data['X'].shape[0]
no_validation = validation['X'].shape[0]
no_test = test['X'].shape[0]
n_epochs = 200

In [10]:
y_train =  np.load('fer2013\\train_label.npy')
from sklearn.utils import class_weight
train_class_weight = class_weight.compute_class_weight('balanced',
                                             np.unique(y_train),
                                             y_train)




In [11]:
def CNN_model(no_classes =7, input_size = 48,  Landmasks_size= Landmasks_size): 
    images_input = Input(shape = (input_size, input_size,1), name='input1')
    x1 = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv11')(images_input)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x1)
    x1 = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv12')(x1)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D((3, 3), strides=(2, 2), name='pool2')(x1)
    x1 = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv13')(x1)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D((3, 3), strides=(2, 2), name='pool13')(x1)
    x1 = Flatten(name='flatten11')(x1)
    x1 = Dense(4096, activation='relu', name='fc11')(x1)
    x1 = Dropout(0.2)(x1)  
    x1 = Dense(1024, activation='relu',name='fc12')(x1)
    x1 = Dropout(0.2)(x1)  
    x1 = BatchNormalization()(x1)
    x1 = Dense(128, activation='relu', name='fc13')(x1)
    images_network = BatchNormalization()(x1)

    ##### hog and landmask network
    landmarks_input = Input(shape = (Landmasks_size,), name='input2')
    x2 = Dense(1024, activation='relu', name='fc21')(landmarks_input)
    x2 = BatchNormalization()(x2)
    x2 = Dense(128, activation='relu', name='fc22')(x2)
    landmarks_network = BatchNormalization()(x2)

    ##### concat
    network = keras.layers.concatenate([images_network, landmarks_network], axis=1)
    output = Dense(no_classes, activation='softmax')(network)

    model = Model(inputs=[images_input, landmarks_input], outputs=output,name='fussed_model')
    return model

model = CNN_model()
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv11 (Conv2D)                 (None, 48, 48, 64)   640         input1[0][0]                     
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 48, 48, 64)   256         conv11[0][0]                     
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 23, 23, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv12 (Co

In [ ]:
opt = Adam(lr=0.001, decay= 10e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
### training  
from keras.callbacks import ModelCheckpoint
# Save check point
filepath = "fer2013.weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose =1, save_best_only = True, mode ='max')
callbacks_list= [checkpoint]

history = model.fit(
    x= [data['X'], data['X2']],
    y = data['Y'],
    batch_size = batch_size,
    epochs = n_epochs,     
#     steps_per_epoch = int(no_training / batch_size), 
    validation_data = ([validation['X'], validation['X2']], validation['Y']), 
#     validation_steps = int(no_validation / batch_size),     
    shuffle=False, 
    class_weight=train_class_weight,
    callbacks = callbacks_list, 
    verbose =2
)

Train on 28709 samples, validate on 3589 samples
Epoch 1/200


In [ ]:
# plot history 
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','valid'], loc= 'upper left' )
plt.show()